In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from datasets import load_dataset, load_metric
import torch
import numpy as np


In [4]:
import wandb

In [5]:
!wandb login


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [9]:
import wandb
import os
os.environ['WANDB_DIR'] = os.getcwd() + '/wandb/'
os.environ['WANDB_CACHE_DIR'] = os.getcwd() + '/wandb/.cache/'
os.environ['WANDB_CONFIG_DIR'] = os.getcwd() + '/wandb/.config/'

import torch
import torch.nn as nn
import torch.optim as optim

# 初始化wandb
wandb.init(
    entity='kaifan-li',
    project="my_project"
)

# 构建模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练循环
for epoch in range(10):
    running_loss = 0.0
    for i in range(100):
        inputs = torch.randn(32, 10)  # 随机生成输入数据
        labels = torch.randn(32, 1)   # 随机生成标签
        
        # 正向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # 记录训练过程和指标
    avg_loss = running_loss / 100
    wandb.log({"epoch": epoch, "loss": avg_loss})


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▅▁▂▅▄▂▂▃
epoch,9
loss,0.99713


In [24]:
import torch
import torch.nn as nn
from transformers import BartModel, BartConfig, BartForConditionalGeneration
from model.prefix_encoder import PrefixEncoder
class BartPrefixForConditionalGeneration(BartForConditionalGeneration):

    def __init__(self, config):
        super().__init__(config)
        # self.model = BartModel(config)
        # self.register_buffer("final_logits_bias", torch.zeros((1, self.model.shared.num_embeddings)))
        # self.lm_head = nn.Linear(config.d_model, self.model.shared.num_embeddings, bias=False)
        
        # MODIFIED
        # Start
        self.config = config
        # self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # End
        
        # https://github.com/huggingface/transformers/issues/4701
        # if we use BartPrefixForConditionalGeneration.from_pretrained() to load the model, 
        # it will not overwrite the pretrained weights of the model
        # Initialize weights and apply final processing
        # self.post_init()
        
        # MODIFIED
        # Start
        for param in self.model.parameters():
            param.requires_grad = False
            
        self.pre_seq_len = config.pre_seq_len
        self.n_layer = config.num_hidden_layers
        self.n_head = config.num_attention_heads
        self.n_embd = config.hidden_size // config.num_attention_heads
        
        self.prefix_tokens = torch.arange(self.pre_seq_len).long()
        self.prefix_encoder = PrefixEncoder(config)
        
        bart_param = 0
        all_param = 0
        
        for name, param in self.model.named_parameters():
            bart_param += param.numel() # numel() returns the total number of elements in the input tensor
        
        for name, param in self.named_parameters():
            all_param += param.numel()
            
        trainable_param = all_param - bart_param
        
        print("Total parameters: {:,}".format(all_param))
        print("Trainable parameters: {:,} {:,%}".format((trainable_param), trainable_param/all_param))
        # End

In [25]:
config = BartConfig.from_pretrained('facebook/bart-large-cnn')
config.pre_seq_len=20
config.prefix_projection=False
bart = BartPrefixForConditionalGeneration(config)
bart

Total parameters: 406,781,952
Trainable parameters: 491,520 0.120831%


BartPrefixForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): 

In [28]:
# 检查模型的参数是否正确加载
state_dict = bart.state_dict()
if any(key.startswith('model.') for key in state_dict.keys()):
    print("BartPrefixForConditionalGeneration 模型成功加载了预训练的权重！")
else:
    print("BartPrefixForConditionalGeneration 模型没有成功加载预训练的权重，请检查模型定义。")

BartPrefixForConditionalGeneration 模型成功加载了预训练的权重！


In [13]:
config = BartConfig.from_pretrained('facebook/bart-large-cnn')
config.pre_seq_len=20

In [1]:
test = [1,2,3,4,5]
test = test[-1:]
test

[5]